# Instructions

For this assignment you will use PyTorch instead of EDF to implement and train neural networks. You should install it (there are multiple tutorials online and it is especially easy if you're using Anaconda). You also have to install the seaborn library which offers some plotting functions that we'll use. Check https://pytorch.org/tutorials/ for some PyTorch tutorials -- this assignment assumes that you know the basics like defining models with multiple modules and coding up functions to do training with PyTorch optimizers.

If you have a nvidia GPU it is a good idea to install CUDA and PyTorch with gpu support so you can run everything on gpu. Another option is to run everything on Google Colab, which offers some free gpu time (check https://colab.research.google.com/ and look for tutorials online).

Unlike previous assignments, in this one you will have to do some writing instead of just coding. Try to keep your answers short and precise, and you are encouraged to write equations if needed (you can do that using markdown cells). You can also use code as part of your answers (like plotting and printing, etc). Blue text indicates questions or things that you should discuss/comment, and there will red "ANSWER (BEGIN)" and "ANSWER (END)" markdown cells to indicate that you should add cells with your writeup between these two. **Make sure not to redefine variables or functions in your writeup, which can change the behavior of the next cells.**

Finally, you might have to do minor changes to the provided code due to differences in python/pytorch versions. You can post on piazza if there's a major, non-trivial change that you had to do (so other students can be aware of it and how to proceed), but for minor changes you should just apply them and keep working on the assignment.

In [ ]:
import torch, math, copy
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from torchvision import datasets, transforms
import torch.nn as nn
import torch.nn.init as init
import torch.nn.functional as F

# Learning rate as temperature in SGD

In this first part you will study how the learning rate acts as temperature in the stationary distribution of the stochastic differential equation (SDE) that approximates SGD. First, recall the SDE derived in the lectures that approximates the continuous-time behavior of SGD:

$$dw = - \nabla L(w) dt + \epsilon \sqrt{\eta dt}, \quad \epsilon \sim \mathcal N(0, \sigma^2), \eta \propto dt$$

where $\eta$ is the learning rate and $\sigma$ captures the noise level of stochastic gradients (i.e. gradients computed from a mini-batch).

Our goal is to sample from the stationary distribution of the above SDE. To do that, we will use Stochastic Gradient Langevin Dynamics (SGLD), an algorithm commonly used to sample from the parameter's posterior -- this is unlike SGD, whose goal is to simply minimize the objective $L$, which can be seen as finding the modes of the parameter's posterior (this will become more clear as you run the experiments and observe the results).

SGLD works by adding an additional source of noise such that the produced iterates are samples of the posterior instead of modes. Unlike the noise in the SDE above (which captures the variance of stochastic gradients), the additional term we will add scales as $\sqrt{\eta}$, meaning that it will dominate the updates if the learning rate $\eta$ is small.

The SGLD update rule for the above SDE is:

$$w_{t+1} = w_t -  \eta \nabla L(w) + \eta \epsilon_1 + \sqrt{\eta} \epsilon_2, \quad \epsilon_1 \sim \mathcal N(0, \sigma^2), \epsilon_2 \sim \mathcal N(0, 1)$$

The update above can be also understood as follows. Let us consider an iterate $w$ and start by taking the SGD update

$$w^{SGD} = w - \eta \nabla \tilde L(w)$$

where $\nabla \tilde L(w)$ is a stochastic estimate of the true gradient $\nabla L(w)$, which we approximate as $\nabla \tilde L(w) \approx \nabla L(w) + \epsilon_1$, where $\epsilon_1$ is normally-distributed with variance $\sigma^2$ that decreases with the mini-batch size (check the lecture slides for more details on this step). This yields the following update:

$$w^{LD} = w - \eta \nabla L(w) + \eta \epsilon_1, \quad \epsilon_1 \sim \mathcal N(0, \sigma^2)$$

Finally, since we want to perform inference and not minimize $L$, we sample $w^{SGLD} \sim \mathcal N(w^{LD}, \eta)$, or equivalently:

$$w^{SGLD} = w - \eta \nabla L(w) + \eta \epsilon_1 + \sqrt{\eta} \epsilon_2, \quad \epsilon_1 \sim \mathcal N(0, \sigma^2), \epsilon_2 \sim \mathcal N(0, 1)$$

You can check Welling & Teh, Bayesian Learning via Stochastic Gradient Langevin Dynamics for more details on SGLD, although this is not needed to complete the assignment: https://www.ics.uci.edu/~welling/publications/papers/stoclangevin_v6.pdf. Another reference is Stochastic Gradient Descent as Approximate Bayesian Inference by Mandt et al.: https://arxiv.org/abs/1704.04289.

The cell below define a 1-dimensional problem characterized by a loss $L(w)$, along with some functions that you'll use for plotting.

In [ ]:
low, high = -60, 60

def L(w):
    return 10*torch.clamp((1 + torch.cos(w) + 0.2*torch.abs(w - 5*math.pi)) / 5.0, max=1.75)

def plot_iters(w_space, loss_space, w_iters=None, loss_iters=None, focus=False):
    plt.xlabel("w")
    plt.ylabel("loss")
    plt.plot(w_space, loss_space)
    if not (w_iters is None or loss_iters is None):
        plt.plot(w_iters, loss_iters, marker='x')
        if focus:
            min_w, max_w = min(w_iters), max(w_iters)
            min_l, max_l = min(loss_iters), max(loss_iters)
            gap_w, gap_l = max_w - min_w, max_l - min_l
            plt.xlim(min_w - gap_w/4, max_w + gap_w/4)
            plt.ylim(min_l - gap_l/4, max_l + gap_l/4)
    plt.tight_layout()
    plt.show()
    plt.clf()
    
def plot_loss(loss_iters):
    plt.xlabel("iter")
    plt.ylabel("loss")
    plt.plot(np.arange(len(loss_iters)), loss_iters)
    plt.tight_layout()
    plt.show()
    plt.clf()
    
def plot_density(w_space, loss_space, w_iters):
    fig, ax1 = plt.subplots()
    ax1.set_xlabel('w')
    ax1.set_ylabel('loss')
    ax1.plot(w_space, loss_space)
    ax2 = ax1.twinx()
    ax2.set_ylabel('p(w)')
    sns.distplot(w_iters, hist = False, kde = True, color='orange',
                 kde_kws = {'shade': True, 'bw' : 0.1, 'clip' : (low, high)})
    fig.tight_layout()
    plt.show()
    plt.clf()

The cell below plots the loss, which was designed to capture some characteristics of deep learning: it contains multiple local minima, is highly non-convex and its gradients provide little to no signal if the parameters become too large.

In [ ]:
w_space = torch.linspace(low, high, 500)
loss_space = L(w_space)
w_space, loss_space = w_space.numpy(), loss_space.numpy()
plot_iters(w_space, loss_space)

An example of how you should aim to answer questions / add discussions is:

**<span style="color:blue">
    Does the above function admit multiple global minima?
</span>**

**<span style="color:red">
    ------------------------------------------------------------------------------- ANSWER (BEGIN) -------------------------------------------------------------------------------
</span>**

No, check the plot produced by the cell below: it has a single global minimum at $w^* \approx 16$ with $L(w^*) \approx 0$, and from the plot above we can see that all other local minima have $L(w) \geq 2$.

In [ ]:
w_space2 = torch.linspace(5, 25, 500)
plot_iters(w_space2.numpy(), L(w_space2).numpy())

**<span style="color:red">
    ------------------------------------------------------------------------------- ANSWER (END) -------------------------------------------------------------------------------
</span>**

Finish implementing SGLD in the cell below. For this experiment you can set $\sigma = 0.1$ as the standard deviation of $\epsilon_1$. Your implementation should work regardless of the dimension of w: you should treat each component as an independent iterate and perform SGLD in parallel for all components of w, i.e. $w_{t+1}[i] = w_t[i] -  \eta \nabla L(w[i]) + \eta \epsilon_1[i] + \sqrt{\eta} \epsilon_2[i]$.

The returned variables w_iters and loss_iters should have shape (T x N), where T is the number of SGLD iterations and N is the number of initial w iterates (the dimension of w), with $w_{iters}[t,i] = w_t[i]$ and $loss_{iters}[t,i] = L(w_t[i])$.

In [ ]:
def SGLD(iters, w, lr):
    w_iters = []
    loss_iters = []
    
    # possibly insert code here (depends on how you implement the function)
    
    for i in range(iters):
        loss = # insert code here
        
        w_iters.append(w.clone().data.numpy())
        loss_iters.append(loss.data.numpy())
        
        # insert code here to perform a SGLD update step

    return np.asarray(w_iters), np.asarray(loss_iters)

Let's check how the iterates look like with learning rates of vastly different magnitudes (also use this as a sanity-check of your implementation), starting from w=30.

In [ ]:
for lr in [0.02, 2.0]:
    w = nn.Parameter(torch.tensor(30.))
    w_iters, loss_iters = SGLD(50, w, lr)
    plot_iters(w_space, loss_space, w_iters, loss_iters)

Now, initialize 10k different values for w from an uniform distribution over $[0,40]$. In the cell below w should be a 10k-dimensional vector and the SGLD call will likely only work if your implementation works for batched w iterates. Also remember that the components should be updated independently, so you should not take the average loss when computing gradients.

After 1500 iterations we will plot the distribution of the last SGLD iterates (one for each of the 10k randomly-initialized values), along with the expected loss per iteration. The first plot shows the loss function (blue, left y-axis) and the distribution of the 10k sampled parameters (orange, with values given by the x-axis and probability in the right y-axis).

In [ ]:
lr = 0.02

w = # insert code here. w should be a 10k-dimensional vector with components sampled from U(0,40)

w_iters, loss_iters = SGLD(1500, w, lr)
plot_density(w_space, loss_space, w_iters[-1])
plot_loss(loss_iters.mean(1))
print("Expected loss: {}".format(loss_iters[-1].mean()))

**<span style="color:blue">
    Check the first of the two plots above. What can you say about the estimated distribution $p(w)$ and the corresponding loss value $L(w)$? Does $p(w)$ look like a Gibbs distribution with energy $E = L$, i.e. $p(w) \propto \exp \left( \frac{-L(w)}{T} \right)$ for some temperature $T$?
</span>**

**<span style="color:red">
    ------------------------------------------------------------------------------- ANSWER (BEGIN) -------------------------------------------------------------------------------
</span>**

**<span style="color:red">
    ------------------------------------------------------------------------------- ANSWER (END) -------------------------------------------------------------------------------
</span>**

Repeat the above but with a learning rate of 1.5.

In [ ]:
lr = 1.5
w = # insert code here, same initialization previously
w_iters, loss_iters = SGLD(1500, w, lr)
plot_density(w_space, loss_space, w_iters[-1])
plot_loss(loss_iters.mean(1))
print("Expected loss: {}".format(loss_iters[-1].mean()))

**<span style="color:blue">
    Compare the two parameter distributions (produced with large/small learning rates) and discuss how and why they differ. What can you say about their temperatures?
</span>**

**<span style="color:red">
    ------------------------------------------------------------------------------- ANSWER (BEGIN) -------------------------------------------------------------------------------
</span>**

**<span style="color:red">
    ------------------------------------------------------------------------------- ANSWER (END) -------------------------------------------------------------------------------
</span>**

Now, let's check what happens if we decay the learning rate of SGLD by a factor of 2 every 500 iterations.

In [ ]:
lr = 1.5
w = # insert code here, same initialization previously
total_loss_iters = []
for k in range(3):
    w_iters, loss_iters = SGLD(500, w, lr)
    plot_density(w_space, loss_space, w_iters[-1])
    lr = lr/2.0
    total_loss_iters.append(loss_iters)
    
total_loss_iters = np.concatenate(total_loss_iters)
plot_loss(total_loss_iters.mean(1))
print("Expected loss: {}".format(total_loss_iters[-1].mean()))

**<span style="color:blue">
    Compare the final expected loss achieved in the three experiments above. Give an explanation for how the parameter density and the loss curve evolved as the learning rate was decayed. Have you seen a curve with a similar shape in the course lectures?
</span>**

**<span style="color:red">
    ------------------------------------------------------------------------------- ANSWER (BEGIN) -------------------------------------------------------------------------------
</span>**

**<span style="color:red">
    ------------------------------------------------------------------------------- ANSWER (END) -------------------------------------------------------------------------------
</span>**

Finally, we will check whether the discrete decays adopted above can be replaced by a more gradual, smooth schedule.

In the cell below implement a schedule $\eta_t = \frac{\eta}{\sqrt t}$ and run SGLD for 1500 iterations, again starting from 10k randomly-initialized parameters.

In [ ]:
lr = # choose a reasonably large value
w = # insert code here, same initialization previously
total_loss_iters = []
for k in range(1500):
    cur_lr = # insert code here
    w_iters, loss_iters = SGLD(1, w, cur_lr)
    total_loss_iters.append(loss_iters)
plot_density(w_space, loss_space, w_iters[-1])

total_loss_iters = np.concatenate(total_loss_iters)
plot_loss(total_loss_iters.mean(1))
print("Expected loss: {}".format(total_loss_iters[-1].mean()))

**<span style="color:blue">
    Try different values for $\eta$ and try to match or outperform the discrete schedule. Were you able to match it? If no, why not?
</span>**

**<span style="color:red">
    ------------------------------------------------------------------------------- ANSWER (BEGIN) -------------------------------------------------------------------------------
</span>**

**<span style="color:red">
    ------------------------------------------------------------------------------- ANSWER (END) -------------------------------------------------------------------------------
</span>**

# The role of the learning rate in deep learning

In this second part you'll check whether the behavior observed above is also observable when training neural networks. In particular, you'll study how the magnitude of the learning rate affects a model's performance, especially when annealed properly.

The following experiments will use a small subset of MNIST. We will use 256 samples for training and 2048 for test.

In [ ]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

train_dataset = datasets.MNIST("data", train=True, download=True, transform=transform)
sampler=torch.utils.data.sampler.SubsetRandomSampler(range(256))
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, sampler=sampler)

test_dataset = datasets.MNIST("data", train=False, download=True, transform=transform)
sampler=torch.utils.data.sampler.SubsetRandomSampler(range(2048))
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=128, sampler=sampler)

Fill the missing code below. In both train_epoch and test, total_correct should be the total number of correctly classified samples, while total_samples should be the total number of samples that have been iterated over.

In [ ]:
def train(epochs, model, criterion, optimizer, train_loader, test_loader, scheduler=None):
    train_errs = []
    test_errs = []
    for epoch in range(epochs):
        train_err = train_epoch(model, criterion, optimizer, train_loader)
        test_err = test(model, test_loader)
        print('Epoch {:03d}/{:03d}, Error: {} || {}'.format(epoch, epochs, train_err, test_err))
        train_errs.append(train_err)
        test_errs.append(test_err)
        if scheduler is not None: scheduler.step()
    return train_errs, test_errs
    
def train_epoch(model, criterion, optimizer, loader):
    total_correct = 0.
    total_samples = 0.
    
    for batch_idx, (data, target) in enumerate(loader):
        
        # insert code to get the model outputs and compute the loss (criterion)

        total_correct += # insert code here
        total_samples += # insert code here
        
        # insert code to update the parameters using optimizer

    return 1 - total_correct/total_samples
    
def test(model, loader):
    total_correct = 0.
    total_samples = 0.

    with torch.no_grad():
        for batch_idx, (data, target) in enumerate(loader):
            
            # insert code to get the model outputs

            total_correct += # insert code here
            total_samples += # insert code here

    return 1 - total_correct/total_samples

def plot_err(train_errs, test_errs):
    plt.xlabel("epochs")
    plt.ylabel("error (%)")
    plt.plot(np.arange(len(train_errs)), train_errs, color='red')
    plt.plot(np.arange(len(test_errs)), test_errs, color='blue')
    plt.legend(['train error', 'test error'], loc='upper right')
    plt.tight_layout()
    plt.show()
    plt.clf()

Implement a CNN with 4 convolutional layers, each with a kernel size of 5, stride of 2 and padding of 2, followed by two fully-connected layers, the first with 2048 hidden units and the last with 10 units. The first conv layer should have w channels and each following conv should double the number of channels. Every layer should be followed by batch norm and then a ReLU activation (except for the last layer).

More specifically, the model should be:

Conv(w, k=5, s=2, p=2) -> BN(w) -> ReLU ->

Conv(2*w, k=5, s=2, p=2) -> BN(w) -> ReLU -> 

Conv(4*w, k=5, s=2, p=2) -> BN(w) -> ReLU -> 

Conv(8*w, k=5, s=2, p=2) -> BN(w) -> ReLU ->

Linear(2048) -> BN(2048) -> ReLU ->

Linear (10)

In [ ]:
class CNN(nn.Module):
    def __init__(self, w):
        super(CNN, self).__init__()
        
        # insert code here 
        
    def forward(self, input):
        
        # insert code here 

        return output

In [ ]:
criterion = torch.nn.CrossEntropyLoss()

We'll first train a CNN with a small learning rate of 0.0002. If your implementation is correct you should be able to reach 0% training error.

In [ ]:
network1 = CNN(128).cuda()
optimizer = torch.optim.SGD(network1.parameters(), lr=0.0002, weight_decay=1e-2, momentum=0.5, nesterov=True)
train_errs, test_errs = train(40, network1, criterion, optimizer, train_loader, test_loader)

In [ ]:
plot_err(train_errs, test_errs)

Now, we'll train the same model but starting with a large learning rate of 1.0, which will be decayed during training. Instantiate a scheduler in the cell below that decays the learning rate by a factor of 10 every 15 epochs: look at the docs of torch.optim.lr_scheduler to see how to do that.

In [ ]:
network2 = CNN(128).cuda()
optimizer = torch.optim.SGD(network2.parameters(), lr=1.0, weight_decay=1e-2, momentum=0.5, nesterov=True)
scheduler = # insert code here 
train_errs, test_errs = train(40, network2, criterion, optimizer, train_loader, test_loader, scheduler)

In [ ]:
plot_err(train_errs, test_errs)

**<span style="color:blue">
    How do the two models compare in terms of train and test performance? Discuss your findings and compare them with your observations with SGLD in the first part of this assignment.
</span>**

**<span style="color:red">
    ------------------------------------------------------------------------------- ANSWER (BEGIN) -------------------------------------------------------------------------------
</span>**

**<span style="color:red">
    ------------------------------------------------------------------------------- ANSWER (END) -------------------------------------------------------------------------------
</span>**

# Estimating a norm-based PAC-Bayes generalization bound

In this last part you'll check whether the two networks trained in the second step have properties that are useful when predicting their generalization capabilities with PAC-Bayes. Recall the general PAC-Bayesian guarantee:

$$L(Q_w) \leq \frac{10}{9} \left( \hat L(Q_w) + \frac{5}{N} \left( \mathcal D(Q_w, P) + \log \frac{1}{\delta} \right)   \right)$$

where $L(Q_w) = \mathbb E_{\epsilon \sim \mathcal N(0, \sigma^2)^d} [L(w + \epsilon)]$ and $\hat L(Q_w) = \mathbb E_{\epsilon \sim \mathcal N(0, \sigma^2)^d} [\hat L(w + \epsilon)]$, with $L, \hat L$ denoting the test and train losses, respectively. Moreover, $Q_w = \mathcal N(w, \sigma^2)^d$ and $\mathcal D(\cdot, \cdot)$ denotes the KL divergence. Let $P = \mathcal N(0, \sigma^2)^d$ (here $d$ denotes the number of parameters of the network).

We will estimate the generalization guarantees given by the above bound as a function of $\sigma$. Note that $L(Q_w)$, $\hat L(Q_w)$ and $\mathcal D(Q_w, P)$ all depend on $\sigma$. You should derive by yourself what $\mathcal D(Q_w, P)$ looks like before moving on.

In the cell below you'll implement the function get_norm, which receives a model as argument and should return the squared $\ell_2$ norm of its parameters, i.e. $\|w\|_2^2 = \sum_i w_i^2$. If you don't know why you'd need this function, go over the lecture slides that cover PAC-Bayes again, or alternatively check this tutorial on PAC-Bayes: https://arxiv.org/pdf/1307.2118.pdf.

In [ ]:
def get_norm(network):
    
    # insert code here 

    return norm

Let's check the squared norm of each of the two networks -- remember that network1 was trained with a small, constant learning rate, while network2 was trained with a large one that was annealed. Both had the same strength for $\ell_2$ regularization.

In [ ]:
norm1 = get_norm(network1)
norm2 = get_norm(network2)
print("Squared l2 norm of model 1: {}".format(norm1))
print("Squared l2 norm of model 2: {}".format(norm2))

Lastly, the function below will estimate $\hat L(Q_w)$ for a given $\sigma$. To do that, you should make 10 copies of the given model, add gaussian noise with variance $\sigma^2$ to its parameters, and compute the average error on the training set.

In [ ]:
def pac_test(model, loader, sigma):
    errs = []

    with torch.no_grad():
        for k in range(10):
            total_correct = 0.
            total_samples = 0.
            
            # depending on your pytorch version the line above might throw an error
            # if this happens, search online on how to deep copy pytorch models in your version
            
            model_clone = copy.deepcopy(model)
            
            # insert code here to add gaussian i.i.d. noise to the model's parameters
            
            for batch_idx, (data, target) in enumerate(loader):
                # insert code here 
                
                # insert code to get the model outputs

                total_correct += # insert code here
                total_samples += # insert code here

            errs.append(1 - total_correct/total_samples)
                
    return sum(errs) / len(errs)

The cells below will compute and print $\hat L(Q_w)$ for 5 different values of $\sigma$ and for each of the two networks trained previously.

In [ ]:
for sigma in [1e-1, 5e-2, 1e-2, 5e-3, 1e-3]:
    pac_test_err = pac_test(network1, train_loader, sigma)
    print("Error of model 1 with sigma={}: {} %".format(sigma, 100*pac_test_err))

In [ ]:
for sigma in [1e-1, 5e-2, 1e-2, 5e-3, 1e-3]:
    pac_test_err = pac_test(network2, train_loader, sigma)
    print("Error of model 2 with sigma={}: {} %".format(sigma, 100*pac_test_err))

**<span style="color:blue">
    Which network maintains a reasonable performance with the largest $\sigma$? Compare with the parameter norms computed previously. Considering the PAC-Bayes guarantee you derived, which network should have better generalization? In the MNIST experiments, which network actually generalized better?
</span>**

**<span style="color:red">
    ------------------------------------------------------------------------------- ANSWER (BEGIN) -------------------------------------------------------------------------------
</span>**

**<span style="color:red">
    ------------------------------------------------------------------------------- ANSWER (END) -------------------------------------------------------------------------------
</span>**